# Overlap-Layout-Consensus Genome Assembly

This notebook demonstrates the usage of the OLC Assembly package for genome assembly from FASTQ files.

In [ ]:
# Import necessary modules
import sys
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Make sure the package is in the Python path
# Change this path to where you've placed the olc_assembly package
sys.path.append('..')

# Import modules from the package
from olc_assembly.fastq_parser import FastqParser
from olc_assembly.overlap_detector import OverlapDetector
from olc_assembly.overlap_graph import OverlapGraph
from olc_assembly.layout_generator import LayoutGenerator
from olc_assembly.consensus_builder import ConsensusBuilder
from olc_assembly.contig_generator import ContigGenerator
from olc_assembly.fasta_writer import FastaWriter
from olc_assembly.assembly_metrics import AssemblyMetrics
from olc_assembly.visualize import Visualizer

## 1. Load and Parse FASTQ Data

Let's start by loading the FASTQ data from one of the synthetic datasets.

In [ ]:
# Path to the FASTQ file
fastq_file = "path/to/reads_b.fastq"

# Parse the FASTQ file
parser = FastqParser(fastq_file)
reads = parser.parse_as_dict()

print(f"Loaded {len(reads)} reads")
print(f"Example read: {list(reads.values())[0].sequence[:50]}...")

## 2. Find Overlaps Between Reads

Next, we'll find the overlaps between reads.

In [ ]:
# Define parameters for overlap detection
min_overlap = 40  # Minimum overlap length
max_error_rate = 0.05  # Maximum error rate (5%)

# Create overlap detector
overlap_detector = OverlapDetector(min_overlap, max_error_rate)

# Compute all pairwise overlaps
overlaps = overlap_detector.compute_all_overlaps(reads)

print(f"Found {len(overlaps)} overlaps between reads")

# Show some example overlaps
if overlaps:
    print("\nExample overlaps:")
    for i, ((read_id1, read_id2), (overlap_len, errors)) in enumerate(list(overlaps.items())[:5]):
        print(f"  {read_id1} -> {read_id2}: {overlap_len} bp with {errors} errors")

## 3. Filter Contained Reads and Get Dovetail Overlaps

Now we'll filter out contained reads and focus on dovetail overlaps.

In [ ]:
# Filter contained reads
contained_reads = overlap_detector.filter_contained_reads(reads, overlaps)
print(f"Filtered out {len(contained_reads)} contained reads")

# Get dovetail overlaps
dovetail_overlaps = overlap_detector.get_dovetail_overlaps(reads, overlaps, contained_reads)
print(f"Found {len(dovetail_overlaps)} dovetail overlaps")

# Filter reads to exclude contained reads
filtered_reads = {read_id: read for read_id, read in reads.items() 
                 if read_id not in contained_reads}

## 4. Build Overlap Graph

Let's build the overlap graph from the filtered reads and dovetail overlaps.

In [ ]:
# Build overlap graph
graph = OverlapGraph()
graph.build_from_overlaps(filtered_reads, dovetail_overlaps)

print(f"Graph built with {len(graph.graph.nodes())} nodes and {len(graph.graph.edges())} edges")

# Remove transitive edges
graph.remove_transitive_edges()
print(f"After transitive reduction: {len(graph.graph.nodes())} nodes and {len(graph.graph.edges())} edges")

# Visualize the graph (only if it's small enough to render)
if len(graph.graph.nodes()) <= 100:  # Don't try to visualize very large graphs
    Visualizer.plot_overlap_graph(graph.graph)
else:
    print("Graph too large to visualize directly.")

## 5. Find Paths and Generate Contigs

Now we'll find paths through the graph and generate contigs.

In [ ]:
# Find non-branching paths
paths = graph.find_nonbranching_paths()
print(f"Found {len(paths)} non-branching paths")

# Generate contigs
contig_generator = ContigGenerator(filtered_reads, dovetail_overlaps)
contigs = contig_generator.generate_contigs(paths)

# Filter out short contigs
min_length = 100
filtered_contigs = contig_generator.filter_contigs(contigs, min_length)

print(f"Generated {len(contigs)} contigs")
print(f"After filtering (min length {min_length}): {len(filtered_contigs)} contigs")

## 6. Analyze Contig Properties

Let's check the properties of our generated contigs.

In [ ]:
# Display some basic info about the contigs
contig_lengths = [len(contig) for contig in filtered_contigs]

print(f"Longest contig: {max(contig_lengths)} bp")
print(f"Shortest contig: {min(contig_lengths)} bp")
print(f"Average contig length: {sum(contig_lengths) / len(contig_lengths):.2f} bp")

# Visualize contig length distribution
Visualizer.plot_contig_length_distribution(filtered_contigs)

## 7. Calculate Assembly Metrics

Now we'll calculate various metrics to evaluate our assembly.

In [ ]:
# Calculate assembly metrics
metrics = AssemblyMetrics.calculate_basic_metrics(filtered_contigs)

# Display metrics in a nice format
print("Assembly Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value}")

## 8. Visualize a Contig Layout

Let's visualize the layout of reads in one of the contigs.

In [ ]:
# Generate layout for the first path
if paths:
    layout_generator = LayoutGenerator(filtered_reads, dovetail_overlaps)
    layout = layout_generator.generate_layout(paths[0])
    
    # Visualize the layout
    Visualizer.plot_layout(layout, filtered_reads)
else:
    print("No paths found to visualize layout.")

## 9. Write Contigs to FASTA

Finally, let's save our assembly to a FASTA file.

In [ ]:
# Write contigs to FASTA file
output_file = "contigs_olc.fasta"
writer = FastaWriter(output_file)
writer.write_contigs(filtered_contigs)

print(f"Contigs written to {output_file}")

## 10. Compare with Reference (if available)

If we have a reference genome, we can compare our assembly to it.

In [ ]:
# Load reference genome if available
reference_file = "path/to/reference_b.fasta"

try:
    with open(reference_file, 'r') as f:
        # Skip header line
        header = f.readline()
        # Read reference sequence
        reference = ''.join(line.strip() for line in f)
        
    print(f"Loaded reference genome: {len(reference)} bp")
    
    # Compare assembly to reference
    comparison_metrics = AssemblyMetrics.compare_to_reference(filtered_contigs, reference)
    
    print("\nComparison to Reference:")
    for metric, value in comparison_metrics.items():
        print(f"  {metric}: {value}")
        
except FileNotFoundError:
    print(f"Reference file {reference_file} not found. Skipping comparison.")

## 11. Experiment with Different Minimum Overlap Lengths

Let's try different minimum overlap lengths and compare the results.

In [ ]:
# Function to run assembly with a specific minimum overlap length
def assemble_with_min_overlap(reads, min_overlap, max_error_rate=0.05, min_length=100):
    # Compute overlaps
    overlap_detector = OverlapDetector(min_overlap, max_error_rate)
    overlaps = overlap_detector.compute_all_overlaps(reads)
    
    # Filter contained reads
    contained_reads = overlap_detector.filter_contained_reads(reads, overlaps)
    dovetail_overlaps = overlap_detector.get_dovetail_overlaps(reads, overlaps, contained_reads)
    filtered_reads = {read_id: read for read_id, read in reads.items() 
                     if read_id not in contained_reads}
    
    # Build graph
    graph = OverlapGraph()
    graph.build_from_overlaps(filtered_reads, dovetail_overlaps)
    graph.remove_transitive_edges()
    
    # Find paths and generate contigs
    paths = graph.find_nonbranching_paths()
    contig_generator = ContigGenerator(filtered_reads, dovetail_overlaps)
    contigs = contig_generator.generate_contigs(paths)
    filtered_contigs = contig_generator.filter_contigs(contigs, min_length)
    
    # Calculate metrics
    metrics = AssemblyMetrics.calculate_basic_metrics(filtered_contigs)
    
    return filtered_contigs, metrics

# Try different minimum overlap lengths
min_overlap_values = [30, 40, 50, 60]
results = {}

for min_overlap in min_overlap_values:
    print(f"\nAssembling with min_overlap={min_overlap}...")
    contigs, metrics = assemble_with_min_overlap(reads, min_overlap)
    results[min_overlap] = (contigs, metrics)
    
    print(f"  Contigs: {metrics['num_contigs']}")
    print(f"  Total length: {metrics['total_length']} bp")
    print(f"  N50: {metrics['n50']}")
    
    # Write to FASTA
    output_file = f"contigs_olc_overlap{min_overlap}.fasta"
    writer = FastaWriter(output_file)
    writer.write_contigs(contigs)
    
# Compare metrics across different minimum overlap values
metrics_list = [results[min_overlap][1] for min_overlap in min_overlap_values]
Visualizer.plot_metrics_comparison(metrics_list, [f"overlap={m}" for m in min_overlap_values])

## 12. Conclusion

We've successfully implemented and tested an Overlap-Layout-Consensus assembly algorithm. The results show how different minimum overlap lengths affect the assembly quality, with trade-offs between contig length, number, and accuracy.